# Parameter Optimization

Find optimal strategy parameters through grid search, walk-forward analysis, and out-of-sample validation.

In [1]:
from datetime import timedelta
from clyptq import CostModel, Constraints
from clyptq.data.loaders.ccxt import load_crypto_data
from clyptq.trading.execution import BacktestExecutor
from clyptq.trading.factors.library.momentum import MomentumFactor
from clyptq.trading.factors.library.volatility import VolatilityFactor
from clyptq.trading.portfolio.constructors import TopNConstructor
from clyptq.trading.strategy.base import SimpleStrategy
from clyptq.trading.optimization import GridSearchOptimizer, WalkForwardOptimizer
from clyptq.trading.optimization.validation import HistoricalSimulator

## 1. Load Data

In [2]:
symbols = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT",
    "ADA/USDT", "AVAX/USDT", "DOGE/USDT", "DOT/USDT", "MATIC/USDT",
]

store = load_crypto_data(symbols=symbols, exchange="binance", timeframe="1d", days=720)
date_range = store.get_date_range()
start = date_range.end - timedelta(days=365)
end = date_range.end

print(f"Optimization period: {start.date()} to {end.date()}")

Optimization period: 2025-01-04 to 2026-01-04


## 2. Define Parameterized Strategy

Momentum strategy with tunable parameters

In [ ]:
class MomentumStrategy(SimpleStrategy):
    def __init__(self, lookback=30, top_n=3):
        self.lookback = lookback
        self.top_n = top_n
        
        factors = [
            MomentumFactor(lookback=lookback),
            VolatilityFactor(lookback=lookback),
        ]
        constraints = Constraints(
            max_position_size=0.40,
            max_gross_exposure=1.0,
            min_position_size=0.10,
            max_num_positions=5,
        )
        super().__init__(
            factors_list=factors,
            constructor=TopNConstructor(top_n=top_n),
            constraints_obj=constraints,
            schedule_str="weekly",
            warmup=lookback + 5,
            name=f"Momentum-{lookback}-{top_n}",
        )

def strategy_factory(params):
    """Factory function that creates strategy from params dict."""
    return MomentumStrategy(lookback=params.get("lookback", 30), top_n=params.get("top_n", 3))

print("Strategy template: Momentum(lookback, top_n)")
print("Parameters to optimize: lookback [10, 20, 30, 40], top_n [2, 3, 4, 5]")

## 3. Grid Search

Test all parameter combinations, find best

In [ ]:
param_grid = {
    "lookback": [10, 20, 30, 40],
    "top_n": [2, 3, 4, 5],
}

cost_model = CostModel(maker_fee=0.001, taker_fee=0.001, slippage_bps=5.0)
executor = BacktestExecutor(cost_model)

optimizer = GridSearchOptimizer(
    strategy_factory=strategy_factory,
    data_store=store,
    executor=executor,
    initial_capital=100000.0,
    scoring_metric="sharpe_ratio",
)

print(f"Testing {len(param_grid['lookback']) * len(param_grid['top_n'])} configurations...")
grid_result = optimizer.search(
    param_grid=param_grid,
    start=start,
    end=end,
    cv_folds=3,
)

print("\nGRID SEARCH RESULTS")
print("=" * 80)
print(f"Best parameters: {grid_result.best_params}")
print(f"Best Sharpe: {grid_result.best_score:.3f}")
print(f"Total combinations tested: {len(grid_result.all_results)}")

## 4. Top Configurations

In [5]:
print("\nTOP 10 CONFIGURATIONS")
print("=" * 80)
print(f"{'Rank':<6} {'Lookback':<10} {'Top N':<8} {'Sharpe':<10}")
print("=" * 80)

top_configs = sorted(
    grid_result.all_results,
    key=lambda x: x["score"],
    reverse=True
)[:10]

for i, config in enumerate(top_configs, 1):
    print(
        f"{i:<6} "
        f"{config['params']['lookback']:<10} "
        f"{config['params']['top_n']:<8} "
        f"{config['score']:>8.3f}"
    )


TOP 10 CONFIGURATIONS
Rank   Lookback   Top N    Sharpe    
1      10         5         155.088
2      10         3          68.982
3      10         4          40.657
4      20         2           0.000
5      20         3           0.000
6      20         4           0.000
7      20         5           0.000
8      30         2           0.000
9      30         3           0.000
10     30         4           0.000


## 5. Walk-Forward Analysis

Validate parameters on rolling windows: train on 60 days, test on next 30 days, repeat.

In [ ]:
wf_optimizer = WalkForwardOptimizer(
    strategy_factory=lambda **params: MomentumStrategy(**params),
    param_grid=param_grid,
    train_days=60,
    test_days=30,
    metric="sharpe_ratio",
    initial_capital=100000.0,
)

print("Running walk-forward (train 60d, test 30d)...")
wf_result = wf_optimizer.optimize(
    data_store=store,
    start=start,
    end=end,
    verbose=True,
)

print("\nWALK-FORWARD RESULTS")
print("=" * 80)
print(f"Number of windows: {len(wf_result.windows)}")
print(f"Mean train Sharpe: {wf_result.avg_train_metric:.3f}")
print(f"Mean test Sharpe: {wf_result.avg_test_metric:.3f}")

# Calculate degradation manually
if wf_result.avg_train_metric > 0:
    degradation = (wf_result.avg_train_metric - wf_result.avg_test_metric) / wf_result.avg_train_metric
    print(f"Degradation: {degradation:.2%}")

# Show most frequent parameter combinations
print("\nMost frequent parameter combinations:")
for params_str, count in sorted(wf_result.best_params_frequency.items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"  {params_str}: {count} windows")

## 6. Out-of-Sample Validation

Test on unseen data to detect overfitting.

In [ ]:
# Split data: train (first 6 months), test (last 6 months)
mid_date = start + timedelta(days=180)

simulator = HistoricalSimulator(
    strategy_factory=strategy_factory,
    data_store=store,
    executor=executor,
    initial_capital=100000.0,
    overfitting_threshold=0.3,
)

print(f"Train: {start.date()} to {mid_date.date()}")
print(f"Test:  {mid_date.date()} to {end.date()}")

oos_result = simulator.run_out_of_sample(
    train_start=start,
    train_end=mid_date,
    test_start=mid_date + timedelta(days=1),
    test_end=end,
    params=grid_result.best_params,
)

print("\nOUT-OF-SAMPLE VALIDATION")
print("=" * 80)
print(f"Train Sharpe:       {oos_result.train_result.metrics.sharpe_ratio:>8.3f}")
print(f"Test Sharpe:        {oos_result.test_result.metrics.sharpe_ratio:>8.3f}")
print(f"Degradation:        {oos_result.degradation_ratio:>8.2%}")
print(f"Stability Score:    {oos_result.stability_score:>8.2%}")
print(f"Overfitted:         {oos_result.is_overfitted}")

if oos_result.is_overfitted:
    print("\nWARNING: Strategy may be overfitted to training data")
else:
    print("\nParameters validated: stable out-of-sample performance")

## 7. Parameter Stability Analysis

Test if nearby parameters also work to avoid parameter overfitting.

In [ ]:
# Test parameters around best
best_lookback = grid_result.best_params["lookback"]
test_grid = [
    {"lookback": best_lookback - 10, "top_n": grid_result.best_params["top_n"]},
    {"lookback": best_lookback, "top_n": grid_result.best_params["top_n"]},
    {"lookback": best_lookback + 10, "top_n": grid_result.best_params["top_n"]},
]

ps_result = simulator.analyze_parameter_stability(
    param_grid=test_grid,
    train_start=start,
    train_end=mid_date,
    test_start=mid_date + timedelta(days=1),
    test_end=end,
)

print("\nPARAMETER STABILITY")
print("=" * 80)
print(f"Tested {len(test_grid)} nearby configurations")
print(f"Robust configs: {len(ps_result.robust_params)}")
print("\nStability scores:")
for config_key, score in sorted(ps_result.stability_scores.items(), key=lambda x: x[1], reverse=True):
    config_idx = int(config_key.split("_")[1])
    params = test_grid[config_idx]
    print(f"  Lookback {params['lookback']}: {score:.2%}")

if len(ps_result.robust_params) >= 2:
    print("\nParameters are stable (nearby values also work)")
else:
    print("\nParameters may be fragile (only one config works well)")

## 8. Summary

**Grid Search**:
- Found best parameters
- Cross-validation prevents overfitting

**Walk-Forward**:
- Validates on rolling windows
- Tests parameter robustness over time

**Out-of-Sample**:
- Critical final validation
- Degradation <30% is acceptable

**Stability**:
- Nearby parameters should also work
- Fragile parameters = overfitting

## Next Steps

- **06_portfolio_optimization.ipynb**: Optimize portfolio construction
- **07_risk_analysis.ipynb**: Comprehensive risk testing
- **08_paper_trading.ipynb**: Deploy to paper trading